Railway Knowledge System
===

Ren Nakagawa

---

## Components
- GPT4o - mini
- RAG with fasis (鉄道に関する技術上の基準を定める省令)

# Set up

In [ ]:
!git clone https://github.com/rakawanegan/pbl-railway-knowledge-system
%cd pbl-railway-knowledge-system
!git checkout tags/v1.0 -b v1.0-branch
%pip install transformers faiss-gpu fugashi ipadic openai >> install.txt

Cloning into 'pbl-railway-knowledge-system'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 210 (delta 102), reused 155 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (210/210), 2.59 MiB | 13.34 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/content/pbl-railway-knowledge-system


In [ ]:
import os
from openai import OpenAI
from tqdm import tqdm
import time
import pandas as pd
from contextlib import contextmanager

from src.main import MakeRailwayKnowledgePromptWithTohokuBERT as MakeRailwayKnowledgePrompt

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [ ]:
# Set up your OpenAI API key
# the api key is nakagawa_dev if you need get another api key, you should see
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def inference(prompt):
    # Make a request to the OpenAI API to use gpt-4-mini
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        # Extract and return the response text using attribute access
        text = dict(dict(dict(response)['choices'][0])['message'])['content']
        return text
    except Exception as e:
        print("An error occurred:", e)
        return None

# Main

In [ ]:
query = "新幹線の騒音防止に関する基準は？"

In [ ]:
with timer("set up"):
    mrp = MakeRailwayKnowledgePrompt(k=2)
with timer("make prompt"):
    prompt = mrp.make_prompt(query)
with timer("inference"):
    answer = inference(prompt)
print('-'*50)
print(prompt)
print('-'*50)
print(answer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

[set up] done in 137 s
[make prompt] done in 0 s
[inference] done in 2 s
--------------------------------------------------
Context: 新幹線の騒音については、新幹線鉄道に係る環境基準(環境省(旧環境庁)告示)によるものとするが、音源対策が騒音の防止または軽減を図る上で最も基本的な施策であり、早期にその達成が困難なことから、環境省(旧環境庁)からの要請により、東海道、山陽、東北、上越の各新幹線については、当面、沿線の住宅の集合度合いに応じ、音源対策により順次75デシベル以下とすることを目標とする。 列車が極めて高い速度（130km/h を超え 160km/h 以下）で通過する踏切道は、踏切遮断機、障害物検知装置（自動車が通行する踏切道に限る。）が設けられていること。また、この場合において、自動車が通行する踏切道は大型自動車が通行しないものであること。ただし、やむを得ず大型自動車が通行する踏切道には、二段型遮断装置、大型遮断装置、オーバーハング型警報装置等、踏切の視認性を高める等の大型自動車の踏切支障を有効に防止するための措置を行うこと。#9.施設及び車両の保全
Query: 新幹線の騒音防止に関する基準は？
Answer:
--------------------------------------------------
新幹線の騒音防止に関する基準は、新幹線鉄道に係る環境基準に基づいています。環境省（旧環境庁）からの指示により、当面は東海道、山陽、東北、上越の各新幹線について、沿線の住宅の集合度合いに応じて音源対策を実施し、騒音を75デシベル以下に抑えることを目標としています。音源対策は騒音の防止または軽減を図るための最も基本的な施策とされています。


In [ ]:
df = pd.read_csv("data/Question.csv")
df.head()

,章,種類,問題,答え
0,1,穴埋問題,鉄道事業者は、施設、車両の設計及び維持管理並びに運行を行うにあたって、____の範囲内で個々...,実施基準
1,1,穴埋問題,新幹線の騒音については、新幹線鉄道に係る環境基準(環境省(旧環境庁)告示)によるものとするが...,75
2,1,短答問題,鉄道事業者は何に基づいて詳細な実施基準を策定する必要がありますか？,実施基準
3,1,短答問題,新幹線の騒音基準は何デシベル以下を目標としていますか？,75デシベル以下
4,1,抽象問題,実施基準を策定する際に鉄道事業者が考慮すべき重要な点は何ですか？,実施基準を策定する際には、鉄道事業者は個々の鉄道事業者の実状を反映し、省令等の項目に対応した...


In [ ]:
df_answer = list()
for i, row in tqdm(df.iterrows(), total=len(df)):
    # break
    row = dict(row)
    query = row["問題"]
    mrp_prompt = mrp.make_prompt(query)
    dict_result = {
        "mrp_prompt": mrp_prompt,
        "answer": inference(mrp_prompt)
    }
    row.update(dict_result)
    df_answer.append(row)
df_answer = pd.DataFrame(df_answer)
df_answer.to_csv("data/Answer.csv", index=False)
df_answer

100%|██████████| 35/35 [01:24<00:00,  2.43s/it]


,章,種類,問題,答え,mrp_prompt,answer
0,1,穴埋問題,鉄道事業者は、施設、車両の設計及び維持管理並びに運行を行うにあたって、____の範囲内で個々の鉄道事業者の実状を反映した詳細な実施基準を策定することとする。,実施基準,Context: １の係員に対する教育及び訓練の実施、適性、知識及び技能の確認は、当該係員の所属する鉄道事業者（係員が所属している事業者が鉄道事業者以外の場合にあっては、当該作業を委託した鉄道事業者。以下同じ。）が実施要領を定めて行うこととする。ただし、教育及び訓練の実施、適性、知識及び技能の確認の管理を当該鉄道事業者が行う場合は、他の者にこれを行わせることができる。 鉄道事業者は、施設、車両の設計及び維持管理並びに運行を行うにあたって、省令の範囲内で個々の鉄道事業者の実状を反映した詳細な実施基準を策定することとする。\nQuery: 鉄道事業者は、施設、車両の設計及び維持管理並びに運行を行うにあたって、____の範囲内で個々の鉄道事業者の実状を反映した詳細な実施基準を策定することとする。\nAnswer:,省令
1,1,穴埋問題,新幹線の騒音については、新幹線鉄道に係る環境基準(環境省(旧環境庁)告示)によるものとするが、音源対策が騒音の防止または軽減を図る上で最も基本的な施策であり、早期にその達成が困難なことから、環境省(旧環境庁)からの要請により、東海道、山陽、東北、上越の各新幹線については、当面、沿線の住宅の集合度合いに応じ、音源対策により順次____デシベル以下とすることを目標とする。,75,Context: 新幹線の騒音については、新幹線鉄道に係る環境基準(環境省(旧環境庁)告示)によるものとするが、音源対策が騒音の防止または軽減を図る上で最も基本的な施策であり、早期にその達成が困難なことから、環境省(旧環境庁)からの要請により、東海道、山陽、東北、上越の各新幹線については、当面、沿線の住宅の集合度合いに応じ、音源対策により順次75デシベル以下とすることを目標とする。 列車が極めて高い速度（130km/h を超え 160km/h 以下）で通過する踏切道は、踏切遮断機、障害物検知装置（自動車が通行する踏切道に限る。）が設けられていること。また、この場合において、自動車が通行する踏切道は大型自動車が通行しないものであること。ただし、やむを得ず大型自動車が通行する踏切道には、二段型遮断装置、大型遮断装置、オーバーハング型警報装置等、踏切の視認性を高める等の大型自動車の踏切支障を有効に防止するための措置を行うこと。#9.施設及び車両の保全\nQuery: 新幹線の騒音については、新幹線鉄道に係る環境基準(環境省(旧環境庁)告示)によるものとするが、音源対策が騒音の防止または軽減を図る上で最も基本的な施策であり、早期にその達成が困難なことから、環境省(旧環境庁)からの要請により、東海道、山陽、東北、上越の各新幹線については、当面、沿線の住宅の集合度合いに応じ、音源対策により順次____デシベル以下とすることを目標とする。\nAnswer:,75デシベル以下とすることを目標とする。
2,1,短答問題,鉄道事業者は何に基づいて詳細な実施基準を策定する必要がありますか？,実施基準,Context: 新幹線の騒音については、新幹線鉄道に係る環境基準(環境省(旧環境庁)告示)によるものとするが、音源対策が騒音の防止または軽減を図る上で最も基本的な施策であり、早期にその達成が困難なことから、環境省(旧環境庁)からの要請により、東海道、山陽、東北、上越の各新幹線については、当面、沿線の住宅の集合度合いに応じ、音源対策により順次75デシベル以下とすることを目標とする。 １の係員に対する教育及び訓練の実施、適性、知識及び技能の確認は、当該係員の所属する鉄道事業者（係員が所属している事業者が鉄道事業者以外の場合にあっては、当該作業を委託した鉄道事業者。以下同じ。）が実施要領を定めて行うこととする。ただし、教育及び訓練の実施、適性、知識及び技能の確認の管理を当該鉄道事業者が行う場合は、他の者にこれを行わせることができる。\nQuery: 鉄道事業者は何に基づいて詳細な実施基準を策定する必要がありますか？\nAnswer:,鉄道事業者は、新幹線の騒音に関する環境基準（環境省（旧環境庁）告示）に基づいて、詳細な実施基準を策定する必要があります。
3,1,短答問題,新幹線の騒音基準は何デシベル以下を目標としていますか？,75デシベル以下,Context: 新幹線の騒音については、新幹線鉄道に係る環境基準(環境省(旧環境庁)告示)によるものとするが、音源対策が騒音の防止または軽減を図る上で最も基本的な施策であり、早期にその達成が困難なことから、環境省(旧環境庁)からの要請により、東海道、山陽、東北、上越の各新幹線については、当面、沿線の住宅の集合度合いに応じ、音源対策により順次75デシベル以下とすることを目標とする。 列車が極めて高い速度（130km/h を超え 160km/h 以下）で通過する踏切道は、踏切遮断機、障害物検知装置（自動車が通行する踏切道に限る。）が設けられていること。また、この場合において、自動車が通行する踏切道は大型自動車が通行しないものであること。ただし、やむを得ず大型自動車が通行する踏切道には、二段型遮断装置、大型遮断装置、オーバーハング型警報装置等、踏切の視認性を高める等の大型自動車の踏切支障を有効に防止するための措置を行うこと。#9.施設及び車両の保全\nQuery: 新幹線の騒音基準は何デシベル以下を目標としていますか？\nAnswer:,新幹線の騒音基準は、音源対策により順次75デシベル以下とすることを目標としています。
4,1,抽象問題,実施基準を策定する際に鉄道事業者が考慮すべき重要な点は何ですか？,実施基準を策定する際には、鉄道事業者は個々の鉄道事業者の実状を反映し、省令等の項目に対応した詳細な基準を策定することが求められます。また、設計方法や検証方法など具体的な数値化を含めた基準を示すことが重要です。さらに、技術的実績や実証データ、理論解析等の客観的な検討方法によって省令への適合を証明することも考慮すべきです。,Context: 新幹線の騒音については、新幹線鉄道に係る環境基準(環境省(旧環境庁)告示)によるものとするが、音源対策が騒音の防止または軽減を図る上で最も基本的な施策であり、早期にその達成が困難なことから、環境省(旧環境庁)からの要請により、東海道、山陽、東北、上越の各新幹線については、当面、沿線の住宅の集合度合いに応じ、音源対策により順次75デシベル以下とすることを目標とする。 無軌条電車以外の鉄道のプラットホームの有効長は、当該プラットホームに発着する列車の最も前方にある旅客車から最も後方にある旅客車（車両に直接接続する形式のホームドアを設ける場合においては、列車の最も前方にある乗降口の前端から最も後方にある乗降口の後端）までの長さのうち最長のものの長さ以上であって、かつ、旅客の安全及び円滑な乗降に支障を及ぼすおそれのないものであること。ただし、当該プラットホームにおける乗降人員が少ない場合であって、列車の運用上やむを得ない場合にあっては、ドア非扱いを行い、かつ、案内放送を行う等の旅客の転落等の危険を防止するための措置を講ずることによりプラットホームの有効長を発着する列車長に対して短くすることができる。\nQuery: 実施基準を策定する際に鉄道事業者が考慮すべき重要な点は何ですか？\nAnswer:,鉄道事業者が実施基準を策定する際に考慮すべき重要な点は以下の通りです：\n\n1. **騒音対策**: 新幹線の運行に伴う騒音については、環境省の基準に基づき、音源対策を講じることが重要です。沿線の住宅の集合度合いに応じて、75デシベル以下を目指す必要があります。\n\n2. **乗客の安全**: プラットフォームの有効長を確保し、乗客の安全で円滑な乗降を促進することが求められます。特にドア非扱いの際には、転落防止に向けた措置や案内放送を行うことが必要です。\n\n3. **環境保護**: 騒音だけでなく、他の環境基準に対する遵守も重要です。環境への影響を最小限に抑えるものであることが求められます。\n\n4. **地域住民との協調**: 沿線地域の住民とのコミュニケーションを図り、騒音やその他の影響についての理解と協力を得ることも重要です。\n\n5. **運行効率**: 列車の運用の効率性を維持しつつ、上記の基準を満たす方法を模索する必要があります。\n\nこれらのポイントを総合的に考慮することで、鉄道事業者は安全で快適な運行を実現しつつ、環境への配慮を行うことができます。
5,1,抽象問題,鉄道の工事における「のり切り」、「切土」、「掘削」、「盛土」、「くい打ち」などの土砂の掘削やその他の作業を行う際、安全対策として何を考慮する必要がありますか？,工事中及び存続期間中に土砂崩壊、陥没、排土すべり出し等によって人に危害を及ぼさないように工事を行うこと。,Context: のり切り、切土、掘削、盛土、くい打ち等の土砂の掘削等を伴う鉄道の工事にあたっては、工事中及び存続期間中に土砂崩壊、陥没、排土すべり出し等によって人に危害を及ぼさないように工事を行うこと。 列車が極めて高い速度（130km/h を超え 160km/h 以下）で通過する踏切道は、踏切遮断機、障害物検知装置（自動車が通